<a href="https://colab.research.google.com/github/asungii/quantization-experiments/blob/main/SmoothQuant_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Installations</h1>

From SmoothQuant repo. Because the specified CUDA compiler is 11.3, and the default Colab CUDA version is as of 7/29/24 12.2, we have to change the CUDA version in Colab.

I found info on how to do this from:

https://medium.com/@ajithkumarv/how-to-modify-cuda-gcc-python-versions-in-colab-584ed4113157

https://github.com/googlecolab/colabtools/issues/4214#issuecomment-1859155789

https://vitalitylearning.medium.com/running-cuda-in-google-colab-525a92efcf75

This is going to be a retry—I'm going to figure out if I can do this submodule work as intended so that this will work.

<h2>Conda Setup</h2>

In [ ]:
!pip install -q condacolab
import condacolab
import os
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


In [ ]:
!conda --version

conda 23.11.0


note: you can only use the base env in condacolab, so it's basically useless.

In [1]:
# setup is from the mit-han-lab GitHub

# these bits are not necessary for Google Colab. conda is getting on my nerves
# !conda create -n smoothquant python=3.8
# !conda activate smoothquant
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install transformers==4.36.0 accelerate datasets zstandard

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1+cu121
    Uninstalling torchvision-0.18.1+cu121:
      Successfully uninstalled torchvision-0.18.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.1+cu121
    Uninstalling torchaudio-2.3.1+cu121:
      Successfully uninstalled torchaudio-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
!git clone https://github.com/mit-han-lab/smoothquant.git

Cloning into 'smoothquant'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 352 (delta 129), reused 113 (delta 97), pack-reused 165
Receiving objects: 100% (352/352), 6.80 MiB | 11.60 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [3]:
%cd /content/smoothquant
!python setup.py install

/content/smoothquant
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tool

In [4]:
import torch

In [5]:
! nvcc --version

# well it's 12.2. that's a bit of an issue

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
!sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
!sudo apt-key add /var/cuda-repo-ubuntu2004-11-3-local/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda-11.3

!nvcc --version

--2024-07-31 15:38:02--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu2004.pin’

cuda-ubuntu2004.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-07-31 15:38:02 (6.24 MB/s) - ‘cuda-ubuntu2004.pin’ saved [190/190]

--2024-07-31 15:38:02--  https://developer.download.nvidia.com/compute/cuda/11.3.0/local_installers/cuda-repo-ubuntu2004-11-3-local_11.3.0-465.19.01-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [ ]:
#!export CUDA_PATH=/usr/local/cuda-11.3/
#!export PATH=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_PATH=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_HOME=/usr/local/cuda-11.3/bin:$PATH
#!export CUDA_HOME=/usr/local/cuda-11.3/

Claude says that the !export doesn't persist outside of the cell. Whoops.

In [7]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.3'
os.environ['PATH'] = '/usr/local/cuda-11.3/bin:' + os.environ['PATH']

# THIS WORKED! YAY!!

! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:15:46_PDT_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0


In [ ]:
print(torch.version.cuda)

11.3


In [8]:
!sudo update-alternatives --display cuda

!sudo update-alternatives --config cuda

cuda - auto mode
  link best version is /usr/local/cuda-12.2
  link currently points to /usr/local/cuda-12.2
  link cuda is /usr/local/cuda
/usr/local/cuda-11.3 - priority 113
/usr/local/cuda-12.2 - priority 122
There are 2 choices for the alternative cuda (providing /usr/local/cuda).

  Selection    Path                  Priority   Status
------------------------------------------------------------
* 0            /usr/local/cuda-12.2   122       auto mode
  1            /usr/local/cuda-11.3   113       manual mode
  2            /usr/local/cuda-12.2   122       manual mode

Press <enter> to keep the current choice[*], or type selection number: 1
update-alternatives: using /usr/local/cuda-11.3 to provide /usr/local/cuda (cuda) in manual mode


Now, we install a GCC version compatible:

In [9]:
# sanity check(?)

!sudo update-alternatives --remove-all gcc
!sudo update-alternatives --remove-all g++

update-alternatives: error: no alternatives for gcc
update-alternatives: error: no alternatives for g++


In [10]:
!sudo apt-get update
!sudo apt-get install build-essential software-properties-common -y
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt-get update
!sudo apt-get install gcc-9 g++-9 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-9 90 --slave /usr/bin/g++ g++ /usr/bin/g++-9
!gcc -v

Get:1 file:/var/cuda-repo-ubuntu2004-11-3-local  InRelease
Ign:1 file:/var/cuda-repo-ubuntu2004-11-3-local  InRelease
Get:2 file:/var/cuda-repo-ubuntu2004-11-3-local  Release [564 B]
Get:2 file:/var/cuda-repo-ubuntu2004-11-3-local  Release [564 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:14 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:15 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubu

In [11]:
# this fixed it again! yay!

os.environ['CC'] = '/usr/bin/gcc-9'
os.environ['CXX'] = '/usr/bin/g++-9'
os.environ['CUDAHOSTCXX'] = '/usr/bin/g++-9'

This works, but you will have to deal with this girthy installation every time you want to use this notebook.

In [12]:
# trying out installing torch-int inside smoothquant to see if that solves the module recognition problem
%cd /content/

/content


In [13]:
! git clone https://github.com/Guangxuan-Xiao/torch-int.git

Cloning into 'torch-int'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1102 (delta 139), reused 118 (delta 118), pack-reused 929
Receiving objects: 100% (1102/1102), 960.91 KiB | 3.03 MiB/s, done.
Resolving deltas: 100% (643/643), done.


In [14]:
%cd /content/torch-int

/content/torch-int


I also had to jump through some hoops trying to figure out this submodule stuff—it is not exactly what the repo says to do

In [15]:
# this is probably where the fix is going to be

! git config submodule.submodules/cutlass.url https://github.com/NVIDIA/cutlass.git
! git submodule update --init --recursive

Cloning into '/content/torch-int/submodules/cutlass'...
Submodule path 'submodules/cutlass': checked out 'c975e2ccbb2dbf13024568b37ffa3498ed0b3aed'


In [16]:
! gcc -v

Using built-in specs.
COLLECT_GCC=gcc
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/9/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:hsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 9.5.0-1ubuntu1~22.04' --with-bugurl=file:///usr/share/doc/gcc-9/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,gm2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-9 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-bootstrap --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-time=yes --with-default-libstdcxx-abi=new --enable-gnu-unique-object --disable-vtable-verify --enable-plugin --enable-default-pie --with-system-zlib --with-target-system-zlib=auto --enable-objc-gc=auto --enable-multiarch --disable-werror --with-arch-32=i686 --with-abi=m64 --with-mu

Yet, when I try to build_cutlass.sh, the CUDA compiler is still identified as 12.2.140. The way to fix this is the cell above, changing the alternatives for CUDA usage

In [17]:
!sudo update-alternatives --display gcc

gcc - auto mode
  link best version is /usr/bin/gcc-9
  link currently points to /usr/bin/gcc-9
  link gcc is /usr/bin/gcc
  slave g++ is /usr/bin/g++
/usr/bin/gcc-9 - priority 90
  slave g++: /usr/bin/g++-9


This still gave me errors for a bit, because you need to use a GCC compatible with the different CUDA version. But even when I installed a new GCC version, it seemed to use it but didn't? Weird.

In [18]:
! source environment.sh
! CC=/usr/bin/gcc-9 CXX=/usr/bin/g++-9 bash build_cutlass.sh   # this one needs GPU

-- CMake Version: 3.30.1
-- The CXX compiler identification is GNU 9.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 11.3.58
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- CUDART: /usr/local/cuda/lib64/libcudart.so
-- CUDA Driver: /usr/local/cuda/lib64/stubs/libcuda.so
-- NVRTC: /usr/local/cuda/lib64/libnvrtc.so
-- Default Install Location: install
-- CUDA Compilation Architectures: 80
-- Enable caching of reference results in conv unit tests
-- Enable rigorous conv problem sizes in conv unit tests
-- Using NVCC flags: -DCUTLASS_TEST_LEVEL=0;-DCUTLASS_TEST_ENABLE_CACHED_RESULTS=1;-DCUTLASS_CONV_UNI

In [19]:
%cd /content/torch-int/

/content/torch-int


Looks like someone here

https://github.com/NVIDIA/FasterTransformer/issues/381

is running into similar issues. After adding:


```
            include_dirs=['torch_int/kernels/include','submodules/cutlass/include'],
```


I am now missing cutlass/util/input something something. Luckily, using

https://nvidia.github.io/cutlass/files.html

I'm able to gain a foggy picture of what the installation was maybe(?) supposed to look like, and I found index_sequence.h inside submodules/cutlass/tools/util/include/cutlass/util/. Turns out that the setup include_dirs needs explicit paths, so I just have to go through and patch up these holes—what I'm confused about is why I have to do this myself?

HOLY CROW IT WORKED!
```
include_dirs=[
    'torch_int/kernels/include',
    'submodules/cutlass/include',
    'submodules/cutlass/tools/util/include',
    'submodules/cutlass/tools'
]
```
This is what did it. Bless up


In [20]:
# gonna try to use the pip as claude recommended

! pip install -e .
#! python setup.py install

Obtaining file:///content/torch-int
  Preparing metadata (setup.py) ... done
  Running setup.py develop for torch_int


In [21]:
import smoothquant
import torch_int

In [23]:
# there was no path for some reason
import sys
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython']


In [24]:
sys.path.append('/content/smoothquant')
sys.path.append('/content/torch-int')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/smoothquant', '/content/torch-int']


Now the issue I'm facing is there not being recognition of smoothquant.opt.

In [ ]:
# from https://stackoverflow.com/questions/487971/is-there-a-standard-way-to-list-names-of-python-modules-in-a-package

import imp
import os
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [ ]:
package_contents('smoothquant')

# it's here, so why isn't it being recognized...?

{'__init__', 'calibration', 'fake_quant', 'opt', 'ppl_eval', 'smooth'}

so now the smoothquant.smoothquant.opt somehow recognizes, but import torch_int doesn't work. Now, after 30 minutes, i realize that maybe I need to say torch-int. But sometimes it's torch_int, sometimes it's torch-int, so what am I supposed to do with that? torch-int is the outer, torch_int is the inner.

Okay, now it worked. Not sure what changed, maybe it was using pip install instead of a manual run.

https://github.com/huggingface/transformers/issues/27701

Now I'm getting AttributeError: type object 'OPTDecoder' has no attribute '_prepare_decoder_attention_mask'
with the smoothquant import. This is because of a refactor that happened in November 2023, so I'm going to change opt.py's _prepare_decoder_attention_mask to _prepare_4d_causal_attention_mask. Line 375 in opt.py.

https://github.com/huggingface/transformers/blob/main/src/transformers/models/opt/modeling_opt.py

Above is also helpful for seeing the documentation of OPTDecoder and why I'm having this issue. I literally don't see _prepare_decoder_attention_mask or _prepare_4d_causal_attention_mask in the OPTDecoder class either, so I'm going to try and revert to the 4.26.0 release.

Okay. Welp. That didn't work.


In [25]:
!pip uninstall -y transformers
!pip cache purge
!pip install transformers==4.34.0 --no-cache-dir

#! pip install --force-reinstall transformers

Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Successfully uninstalled transformers-4.36.0
Files removed: 248
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 183.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 325.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 274.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.23.5
    Uninstalling huggingface-hub-0.23.5:
      Successfully uninstalled huggingface-hub-0.23.5
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conf

Okay, it kind of looks like that when I have transformers 4.43.3 (latest), there's no definition for this _prepare attribute, but when I had 4.26.0 there was one. Let me check that I'm not tripping out.

Okay, based on this https://github.com/huggingface/transformers/issues/27701 GitHub error, I've decided to use transformers v4.34.0, which seemed to work, because it gave me a different error:

```
RuntimeError: Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (/usr/local/lib/python3.10/dist-packages/huggingface_hub/__init__.py)
```

Which seems to coincide with a previous error up above when I uninstalled transformers 4.36 and replaced it with 4.34:

```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires huggingface-hub>=0.21.2, but you have huggingface-hub 0.17.3 which is incompatible.
Successfully installed huggingface-hub-0.17.3 tokenizers-0.14.1 transformers-4.34.0
```

So I'm going to now upgrade huggingface-hub to 0.21.2.

In [28]:
!pip install huggingface-hub==0.21.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.21.2 which is incompatible.


IT WORKED. YES!!!!! THE MODEL IS FINALLY INSTALLING!!!!! Victory from the basement of IIT. I'm committing to git rn.

In [ ]:
# for some reason, smoothquant.smoothquant.opt works?
import transformers
print(transformers.__version__)

# yet, when i print the version, it seems like transformers should be able to reference it.
# ??

import torch_int
from smoothquant.opt import Int8OPTForCausalLM
model = Int8OPTForCausalLM.from_pretrained("mit-han-lab/opt-30b-smoothquant")

4.34.0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/115k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/9.80G [00:00<?, ?B/s]